In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import dashi
dashi.visual()

# Load and process some slow signal data

In [ ]:
from ssm.core import pchain
from ssm.core.util_pmodules import Aggregate,SimpleInjector
from ssm.pmodules import *

# initializing our process chain
data_proc = pchain.ProcessingChain()

reader = Reader('/home/sflis/CTA/projects/SSM-analysis/data/astri_onsky/d2019-05-08/Run13312.hdf5')
data_proc.add(reader)
#This module removes incomplete frames and marks bad and unstable pixels
frame_cleaner = PFCleaner()
data_proc.add(frame_cleaner)
# A simple flat field computation based on the first 7000 frames
sff = SimpleFF(0,7000)
data_proc.add(sff)

#The Aggregate module collects the computed object from the frame
aggr = Aggregate(["raw_resp","simple_ff"])
data_proc.add(aggr)

# Simple visualization of the chain
print(data_proc)

#Execute the chain
data_proc.run()


# Plot the processed data

In [ ]:
from CHECLabPy.plotting.camera import CameraImage

data = aggr.aggr["raw_resp"][0]
ff = aggr.aggr['simple_ff'][0]

time_averaged = np.mean(data.data,axis=0)
camera_averaged = np.nanmean(data.data-ff,axis=1)
#A time series plot
plt.figure()
plt.plot(data.time-data.time[0],camera_averaged)

plt.xlabel('Time since run start (s)')
plt.ylabel("Average amplitude (mV)")

#Different average camera images
camera = CameraImage(data.xpix, data.ypix, data.size)
camera.image = time_averaged
camera.add_colorbar('Amplitdue (mV)')
camera.ax.set_title('Raw Data')

camera = CameraImage(data.xpix, data.ypix, data.size)
camera.image = time_averaged -ff
camera.add_colorbar('Amplitdue (mV)')
camera.ax.set_title('Flat fielded')

camera = CameraImage(data.xpix, data.ypix, data.size)
camera.image = ff
camera.add_colorbar('Amplitdue (mV)')
camera.ax.set_title('Flat field coefficents')


# More processing

In [ ]:
from ssm.core.util_pmodules import SimpleInjector
from ssm.pmodules import *
from ssm import pmodules
proc_chain = pchain.ProcessingChain()
#Copy data so that we do not overwrite the original data
ffdata = copy.deepcopy(data)
# apply the flatfielding
ffdata.data -=ff
#The Simple injector just creates a frame with the content of the input dictionary
injector =  SimpleInjector({'data':ffdata})
proc_chain.add(injector)
#Smoothing the signal maybe not so useful right now
smooth =SmoothSlowSignal()
proc_chain.add(smooth)
#Finds hotspot clusters
clust = pmodules.ClusterCleaning(1.,0.9)
clust.in_data = 'data'#smooth.out_data
proc_chain.add(clust)

#The Aggregate module collects the computed object from the frame
#We want the clusters and the smooth data
aggr = Aggregate(["clusters","smooth_data"])
proc_chain.add(aggr)

print(proc_chain)

In [ ]:
proc_chain.run()

In [ ]:
clusters = aggr.aggr['clusters'][0]
smooth_data = aggr.aggr['smooth_data'][0]

# Plotting camera image with highlited clusters

In [ ]:
frame_n = 1000
plt.set_cmap('Greys_r')

#Different average camera images
camera = CameraImage(smooth_data.xpix, smooth_data.ypix, smooth_data.size)
im = copy.deepcopy(smooth_data.data[frame_n])
im[np.isnan(im)] = np.nanmean(im)
camera.image = im



camera.add_colorbar('Amplitdue (mV)')
camera.ax.set_title('Raw Data, {} clusters'.format(len(clusters[frame_n])))

camera.highlight_pixels([item for sublist in clusters[frame_n] for item in sublist],color='r')
camera.set_limits_minmax(-20,100)

# Make movie

In [ ]:
from ssm.putils import make_ssmovie
#for each frame put all the cluster pixels in one list
c = []
for cc in clusters:
    c.append([item for sublist in cc for item in sublist])
#We only make the movie with every 200th frame
sel = range(0,len(smooth_data.data),200)
make_ssmovie(smooth_data,sel,c)


# Watch movie

In [ ]:
from IPython.display import Video
Video("out.mp4",width=1000)

In [1]:
from ssm.putils import make_ssmovie
make_ssmovie?